In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install anthropic
!pip install ratelimiter
!pip install python-dotenv

In [ ]:
import subprocess
import whisper
from datetime import timedelta
from srt import Subtitle
import srt
import logging

# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'video.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("medium")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

# SRTファイル読み込み (エンコーディング指定)
with open('output.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

In [ ]:
from datetime import timedelta
from srt import Subtitle
import srt
import logging
import anthropic
import re
from ratelimiter import RateLimiter
from dotenv import load_dotenv
import os

load_dotenv()  
api_key = os.getenv("CLAUDE_KEY")  

#  ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# OpenAIのAPIキーを設定します
client = anthropic.Anthropic(
    api_key= api_key
)

rate_limiter = RateLimiter(max_calls=50, period=60)

def claude_translate(text):
    try:
        with rate_limiter:
            message = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=1000,
                temperature=0.2,
                system="You are an expert in Japanese-English translator",
                messages=[
                    {"role": "user", "content":f"以下の文を日本語に翻訳してください。翻訳した文章のみを返してください:\n{text}"}
                ]
            )
            translated_text = message.content[0].text
            cleaned_text = re.sub(r'^.*?:\s*\n?', '', translated_text, flags=re.MULTILINE)
            cleaned_text = re.sub(r'^.*?(日本語|以下|翻訳).*?\n', '', cleaned_text, flags=re.MULTILINE)
            return cleaned_text.strip()
    except Exception as e:
        logging.error(f"Translation failed for '{text}': {e}")
        return text

def read_srt(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        srt_data = file.read()
    return list(srt.parse(srt_data))

def translate_subtitles(subtitles):
    for subtitle in subtitles:
        original_text = subtitle.content
        translated_text = claude_translate(original_text)
        subtitle.content = translated_text
        logging.info(f"Translated: {original_text} -> {translated_text}")

def write_srt(output_file, subtitles):
    translated_srt_content = srt.compose(subtitles)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(translated_srt_content)

def translate_srt(input_file, output_file):
    subtitles = read_srt(input_file)
    translate_subtitles(subtitles)
    write_srt(output_file, subtitles)

translate_srt('output2.srt', 'translated_file.srt')


In [ ]:
# 改行関数 (句読点・長さに応じて改行)
def add_line_breaks(text, max_length=20, max_lines=3):
    result = ""
    current_line = ""
    lines = []

    for char in text:
        if char == "、" or char == "。":
            current_line += char
            if len(current_line) > max_length or char == "。":
                lines.append(current_line)
                current_line = ""
        else:
            current_line += char

    if current_line:
        lines.append(current_line)

    # 行数を制限する
    while len(lines) > max_lines:
        line1 = lines.pop(0)
        line2 = lines.pop(0)
        combined_line = line1 + line2
        lines.insert(0, combined_line)

    return "\n".join(lines)

# 改行関数 (SRT形式保持)
def add_line_breaks_srt(srt_text):
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加 (字幕間の空白)
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 改行されたテキストを取得
with open('translated_file.srt', 'r', encoding='utf-8') as file:
    srt_text = file.read()

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)

# ファイルへの書き込み (エンコーディング指定)
with open("translated_file_result.srt", "w", encoding='utf-8') as file:
    file.write(result)

In [ ]:
import subprocess

subprocess.run([
    'ffmpeg',
    '-hwaccel', 'auto',  # ハードウェアアクセラレーションを有効にする
    '-i', 'video.mp4',
    '-vf', "subtitles=translated_file_result.srt:force_style='FontName=Helvetica,FontSize=11'",
    '-c:v', 'h264_amf',  # AMDのハードウェアエンコーダーを利用
    '-c:a', 'copy',  # オーディオストリームをコピーする
    'video_final.mp4'
])